In [1]:
import xgboost as xgb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from numba import jit
import math
import mlflow

import xgboost as xgb

from libs.metrics import smape_fast
from libs.data_engineering import prepareDataXGBoost
import time

/Users/hugotodeschini/Documents/Benchmark forecasting model/ArtefactBenchmarkForecastingModel/libs/data_engineering.py:9: DeprecationWarning: invalid escape sequence \.
  '(.+)_([a-z][a-z]\.)?((?:wikipedia\.org)|(?:commons\.wikimedia\.org)|(?:www\.mediawiki\.org))_([a-z_-]+?)$')


In [2]:
size_data = 1000
encoding = "oneHotEncoding"#oneHotEncoding, label
model = 'XGBoost'

In [3]:
train_1 = pd.read_csv('../Data/train_1.csv')
train_1_reduce = train_1[0:size_data]

In [4]:
tps1= time.clock()
data = prepareDataXGBoost(train_1_reduce, encoding)
tps2= time.clock()
print("Temps d'exécution total:" + str(tps2-tps1) + " secondes")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Temps d'exécution total:2.044507000000001 secondes


In [5]:
data.head()

Visitors  \
Page                                    Dates                  
100毛_zh.wikipedia.org_all-access_spider 2015-07-01      14.0   
                                        2015-07-02       6.0   
                                        2015-07-03      14.0   
                                        2015-07-04      10.0   
                                        2015-07-05       5.0   

                                                    Visitors_shift_1  \
Page                                    Dates                          
100毛_zh.wikipedia.org_all-access_spider 2015-07-01               NaN   
                                        2015-07-02              14.0   
                                        2015-07-03               6.0   
                                        2015-07-04              14.0   
                                        2015-07-05              10.0   

                                                    Visitors_shift_2  \
Page                                    Dates                          
100毛_zh.wikipedia.org_all-access_spider 2015-07-01               NaN   
                                        2015-07-02               NaN   
                                        2015-07-03              14.0   
                                        2015-07-04               6.0   
                                        2015-07-05              14.0   

                                                    Visitors_shift_3  \
Page                                    Dates                          
100毛_zh.wikipedia.org_all-access_spider 2015-07-01               NaN   
                                        2015-07-02               NaN   
                                        2015-07-03               NaN   
                                        2015-07-04              14.0   
                                        2015-07-05               6.0   

                                                    Visitors_shift_4  \
Page                                    Dates                          
100毛_zh.wikipedia.org_all-access_spider 2015-07-01               NaN   
                                        2015-07-02               NaN   
                                        2015-07-03               NaN   
                                        2015-07-04               NaN   
                                        2015-07-05              14.0   

                                                    Visitors_shift_5  \
Page                                    Dates                          
100毛_zh.wikipedia.org_all-access_spider 2015-07-01               NaN   
                                        2015-07-02               NaN   
                                        2015-07-03               NaN   
                                        2015-07-04               NaN   
                                        2015-07-05               NaN   

                                                    Visitors_shift_6  \
Page                                    Dates                          
100毛_zh.wikipedia.org_all-access_spider 2015-07-01               NaN   
                                        2015-07-02               NaN   
                                        2015-07-03               NaN   
                                        2015-07-04               NaN   
                                        2015-07-05               NaN   

                                                    Visitors_shift_7  \
Page                                    Dates                          
100毛_zh.wikipedia.org_all-access_spider 2015-07-01               NaN   
                                        2015-07-02               NaN   
                                        2015-07-03               NaN   
                                        2015-07-04               NaN   
                                        2015-07-05               NaN   

                                                    Visitors_shift_9

In [6]:
data.shape

(550000, 12)

In [7]:
data.columns

Index(['Visitors', 'Visitors_shift_1', 'Visitors_shift_2', 'Visitors_shift_3',
       'Visitors_shift_4', 'Visitors_shift_5', 'Visitors_shift_6',
       'Visitors_shift_7', 'Visitors_shift_90', 'agent_all-access_spider',
       'site_wikipedia.org', 'country_zh'],
      dtype='object')

In [8]:
metrics_df = pd.DataFrame(columns=['RMSE','SMAPE','MAPE'])

X = data.drop('Visitors', axis = 1).values
y = data['Visitors'].values

tscv = TimeSeriesSplit()
print(tscv)

for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    paramSearch = {'max_depth':[2,4,6], 'eta':[1]}
    for max_depth in paramSearch['max_depth']:
        print(max_depth)
        for eta in paramSearch['eta']:
            print(eta)
            with mlflow.start_run():
                num_round = 2
                param = {'max_depth': max_depth, 'eta': eta, 'objective=reg' :'tweedie'}
                bst = xgb.train(param, dtrain, num_round)
                preds = bst.predict(dtest)

                rmse = np.sqrt(mean_squared_error(y_test, preds))
                smape = smape_fast(y_test, preds)
                mape = np.mean(np.abs(preds - y_test)/np.abs(y_test))
                print("Test RMSE: %.3f" % rmse)
                print("Test SMAPE: %.3f" % smape)
                print("Test MAPE: %.3f" % mape)
                new_metrics = pd.DataFrame([(rmse,smape,mape)], columns = ['RMSE','SMAPE','MAPE'])
                metrics_df= metrics_df.append(new_metrics, ignore_index=True)
                
                mean_RMSE = metrics_df['RMSE'].mean()
                sum_RMSE = metrics_df['RMSE'].sum()
                mean_SMAPE = metrics_df['SMAPE'].mean()
                mean_MAPE = metrics_df['MAPE'].mean()
                
                mlflow.log_param("encoding",encoding)
                mlflow.log_param("model",model)
                mlflow.log_param("max_depth",param['max_depth'])
                mlflow.log_param("eta",param['eta'])
                mlflow.log_param("objective",param['objective=reg'])
                mlflow.log_metric('mean_RMSE', mean_RMSE)
                mlflow.log_metric('sum_RMSE', sum_RMSE)
                mlflow.log_metric('mean_SMAPE', mean_SMAPE)
                mlflow.end_run()

TimeSeriesSplit(max_train_size=None, n_splits=5)
TRAIN: [    0     1     2 ... 91667 91668 91669] TEST: [ 91670  91671  91672 ... 183333 183334 183335]
2
1
[17:48:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide


Test RMSE: 1203.233
Test SMAPE: 71.438
Test MAPE: inf
4
1
[17:48:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide


Test RMSE: 1238.879
Test SMAPE: 70.860
Test MAPE: inf
6
1
[17:48:07] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




...

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide


Test RMSE: 56.689
Test SMAPE: 80.959
Test MAPE: inf
4
1
[17:48:16] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide


Test RMSE: 65.205
Test SMAPE: 68.338
Test MAPE: inf
6
1
[17:48:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { objective=reg } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Test RMSE: 44.565
Test SMAPE: 55.901
Test MAPE: inf


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide
